## Multi-Accent and Multi-Lingual Voice Clone Demo with MeloTTS

In [1]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import ToneColorConverter

ckpt_converter = 'checkpoints_v2/converter'
device = "cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs_v2'

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)

/home/enrique/miniconda3/envs/openvoice/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.

Loaded checkpoint 'checkpoints_v2/converter/checkpoint.pth'
missing/unexpected keys: [] []


### Obtain Tone Color Embedding
We only extract the tone color embedding for the target speaker. The source tone color embeddings can be directly loaded from `checkpoints_v2/ses` folder.

In [2]:

#reference_speaker = 'resources/example_reference.mp3' # This is the voice you want to clone
reference_speaker = 'resources/training_enrique.mp3' # This is the voice you want to clone
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, vad=False)

OpenVoice version: v2


Estimating duration from bitrate, this may be inaccurate


#### Use MeloTTS as Base Speakers

MeloTTS is a high-quality multi-lingual text-to-speech library by @MyShell.ai, supporting languages including English (American, British, Indian, Australian, Default), Spanish, French, Chinese, Japanese, Korean. In the following example, we will use the models in MeloTTS as the base speakers. 

In [3]:
from melo.api import TTS

texts = {
    'EN_NEWEST': """Don’t be an asshole! 

Consider closely every single opportunity in your career. You owe your current employer nothing other than respect and gratitude.

Period!

Initially, I wanted to title this lesson «Unlocking Growth: The Power of Opportunity Cost in Your Career Journey,» but finally, I decided to be frank with you and take this subtle license with a non-polite word that maybe would move you (that’s our intention, really) to take action about this critical piece of advice we are just about to share with you here. 

In this first lesson to notably improve your life and your tech career as a data engineer or cloud architect, I want to share a personal story with you.

It's that kind of story you don’t usually pay attention to. 

It is a story about loyalty and belonging.
A story which I think you can take to your own personal or professional life and look at opportunities with a different pair of glasses, particularly a situation that maybe you have already experienced and, if not, it’s sure you will confront sometime sooner than later.

**I’m talking about understanding opportunity cost.**

Not just understanding the concept but how to tackle this concept with a good mindset can change your professional career to other good opportunities representing a big push in experience, projects, money… you name it!

In simple terms, opportunity cost is what you forego when you choose one option over another – the 'cost' of missing out on the second-best choice. I'll share this concept through my own journey, showing you how it played a pivotal role 
in my career decisions.

Hey there! I'm Enrique Catalá, a seasoned data architect with over two decades of experience in the field. Miguel and I were integral to a leading consulting firm specializing in Microsoft's data platform for over a decade. 

God Damm! We don’t want to look back!
In the early days of Azure, we led the change for them, shifting from on-premises solutions to pioneering cloud-first strategies. 

I also was recognized as Microsoft MVP 10 years in a row! 

There I was, at what seemed like the zenith of my career, steering significant projects at a leading consulting firm. 
Recognition? Check. 

Big projects? Check. 

But here's the twist - comfort can be a deceptive trap.

A stopper for something even… bigger!
I still remember the day I decided to respond to a recruiter on LinkedIn. 
The reason? 

It involves 'pigs and flower petals' - quite an exciting story, but I'll save it for another time. This could be a good reason to stay in our newsletter, and why not? Maybe you decide to accelerate your data or cloud career in one of our mentoring programs, working with us on your own project.

Nevertheless… the story for today…

It turned out that this recruiter offered me an exciting position at a big insurance firm in the Netherlands. Curiosity got the best of me, and I went for it. 

**This decision made me confront the opportunity cost.**

—Staying in one place meant missing out on innovative projects and higher salaries elsewhere. This is normal, as our brains don’t like changes or uncertainty.

After that interview, more followed, and what started as mere curiosity unveiled a shocking truth: **There are always more —let’s say—challenging projects and higher salaries out there**, a stark reminder of the opportunities I missed.

I didn't pay attention to the market during my last three years at that consulting firm, so I spent the opportunity cost. 

And let me tell you a secret: **There's a shortage of highly qualified professionals in data platform roles**

-That’s not an opinion.

That’s a fact. 

And I like the facts.

As an engineer, you probably are used to liking facts, too.

Accept it and make the most out of it.
Once I embraced this reality, just two months later, I switched to where I'm working now. A place where I've rediscovered that sense of challenge I felt in my early days, but this time, thankfully, it comes with a great paycheck.

And yes, I know… I had to confront that fear to…

What if I change for a worse job?

There is a possibility, that is right! But again, don’t lose the point… You’ve chosen with data on your hands, and if the possibility finally is not what they sold you, repeat the process.
This is another fact… our tech world is neither ending tomorrow nor the day after tomorrow. 

Embrace that.

And always look for a better opportunity.
Okay, so why are you bringing up the opportunity cost? 

Well, it's because time went by, and Miguel was still at that consulting firm I left a while ago... and as time passed, one day after a few months, we sat down to talk about the old times.

It turns out he had the same wake-up call and told me he was also in several rounds of interviews... but the best part isn't that... 

The best part? 

is that now we're working together again.

Have you considered what opportunities you might miss in your current role?  

Don´t wait like Miguel and me for the next ample opportunity to find you; start seeking it out today. At least… leave yourself to be “wanted”.

In our next lesson 2 of this gift, «3 Cloud Data Ninjas professional tips» that you have got free by subscribing, we are going to tell you about a strange habit that, as simple as it seems, many, many people find so difficult to follow and that, with just a bit of practice and some ideas —that we will share with you— if you put it into practice and maintain it,  you will be able to make a quantitative and decisive leap for your career and your professional value so that they love you more so that they love you... 

But that’d be in lesson 2.

Stay tuned!
    """
}


src_path = f'{output_dir}/tmp.wav'

# Speed is adjustable
speed = 1.0

for language, text in texts.items():
    model = TTS(language=language, device=device)
    speaker_ids = model.hps.data.spk2id
    
    for speaker_key in speaker_ids.keys():
        speaker_id = speaker_ids[speaker_key]
        speaker_key = speaker_key.lower().replace('_', '-')
        
        source_se = torch.load(f'checkpoints_v2/base_speakers/ses/{speaker_key}.pth', map_location=device)
        model.tts_to_file(text, speaker_id, src_path, speed=speed)
        save_path = f'{output_dir}/output_v2_{speaker_key}.wav'

        # Run the tone color converter
        encode_message = "@MyShell"
        tone_color_converter.convert(
            audio_src_path=src_path, 
            src_se=source_se, 
            tgt_se=target_se, 
            output_path=save_path,
            message=encode_message)

 > Text split to sentences.
Don't be an asshole! Consider closely every single opportunity in your career. You owe your current employer nothing other than respect and gratitude. Period! Initially, I wanted to title this lesson Unlocking Growth: The Power of Opportunity Cost in Your Career Journey,
but finally, I decided to be frank with you and take this subtle license with a non-polite word that maybe would move you that's our intention, really to take action about this critical piece of advice we are just about to share with you here. Stay tuned!
 > ===========================


  0%|          | 0/2 [00:00<?, ?it/s]Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 2/2 [00:15<00:00,  8.00s/it]
